In [ ]:
# importing the needed libraries
import tweepy
import requests
from bs4 import BeautifulSoup

# keys and tokens
consumer_key = 'xxxxxxxxxxxxxxxxxxxxxxx'
consumer_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
access_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
access_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

# authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

# API
api = tweepy.API(auth)

# assigning the article url to a variable
article_url = 'https://www.theverge.com/tech'

# getting the html with the requests library
response = requests.get(article_url)

# creating a beautiful soup object with the text gotten from the response of the request
soup = BeautifulSoup(response.text, 'html.parser')

# fetching the title of the first article
article = soup.find('a', class_='after:absolute')
title = article.find_next('a').text

# the body
body = soup.find_all('p')[1].text

# the link to the article
link = article.find_next('a')['href']

# the variable containing the content of the tweet
tweet = '#DailyTechNews: ' + title + ' - ' + body + ' Read More Here -> https://theverge.com' + link

# the image to be included in the tweet
image = soup.find_all('img')[1]['src']
image_url = image
response = requests.get(image_url)

# saving the image to a file
with open(image.split('/')[-1], "wb") as f:
    f.write(response.content)

# uploading the image to get the media id
media = api.media_upload(image.split('/')[-1])

# updating the status
if len(tweet) < 278:
    api.update_status(status=tweet, media_ids=[media.media_id])
else:
    first_tweet = api.update_status(status=tweet[:270]+'...', media_ids=[media.media_id])
    second_tweet = api.update_status(status='...'+tweet[270:],
                                         in_reply_to_status_id=first_tweet.id, 
                                         auto_populate_reply_metadata=True)